In [ ]:
import numpy as np
import scipy.io as sio 
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("input_image_64.jpeg")
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
            img[i, j, 0], img[i, j, 2] =  img[i, j, 2], img[i, j, 0]       
plt.imshow(img)

In [ ]:
in_img = np.reshape(img[:, :, 0], (img.shape[0], img.shape[1], 1))
plt.imshow(in_img[:, :, 0], cmap='gray', vmin=0, vmax=255)

In [ ]:
def padding(input_slice, pad):
    (H, W) = input_slice.shape
    if pad > 0:
        output_slice = np.zeros((H + 2 * pad, W + 2 * pad))
        output_slice[pad : H + pad, pad : W + pad,] = input_slice
        output_slice[pad : H + pad, 0 : pad]               = [[input_slice[i, 0] for j in range(pad)]  for i in range(H)]     # the leftmost two col
        output_slice[H + pad : H + 2 * pad, pad : W + pad] = [[input_slice[H - 1, j] for j in range(W)]  for i in range(pad)] # the top two rows
        output_slice[pad : H + pad, W + pad : W + 2 * pad] = [[input_slice[i, W - 1] for j in range(pad)]  for i in range(H)] # the rightmost two col     
        output_slice[0 : pad, pad : W + pad]               = [[input_slice[0, j] for j in range(W)]  for i in range(pad)]     # the bottom two rows    
        output_slice[0 : pad, 0 : pad]                             = input_slice[0, 0]          # down left corner (4 points)               
        output_slice[H + pad : H + 2 * pad, 0 : pad]               = input_slice[H - 1, 0]      # up left corner   (4 points)                    
        output_slice[H + pad : H + 2 * pad, W + pad : W + 2 * pad] = input_slice[H - 1, W - 1]  # up right corner  (4 points)                
        output_slice[0 : pad, W + pad : W + 2 * pad]               = input_slice[0, W - 1]      # down right corner(4 points)  
    
        return output_slice
    
    return input_slice

In [ ]:
padding_image_1 = np.pad(in_img[:, :, 0], ((5, 5), (5, 5)), mode = 'edge')
plt.imshow(padding_image_1, cmap='gray', vmin=0, vmax=255)

In [ ]:
padding_image_2 = padding(in_img[:, :, 0], 5)
plt.imshow(padding_image_2, cmap='gray', vmin=0, vmax=255)

In [ ]:
def conv_channel(prev_output_channel, flt):
    stride = 1
    f      = flt.shape[0]
    pad    = int((f - 1) * .5)
    (H, W) = prev_output_channel.shape
    paded_channel  = padding(prev_output_channel, pad)
    output_channel = np.zeros((H, W));
    for i in range(H):
        for j in range(W):
            idx = i * stride;
            idy = j * stride;
            output_channel[i, j] += np.sum(np.multiply(paded_channel[idx : idx + f, idy : idy + f], flt))
            
    return output_channel

In [ ]:
def deconv_channel(prev_output_channel, flt, stride):
    f      = flt.shape[0];
    pad    = 1
    (H, W) = prev_output_channel.shape
    paded_channel = padding(prev_output_channel, pad) 
    (H_p, W_p)   = paded_channel.shape;
    (H_e, W_e)          = (H_p * stride, W_p * stride) 
    expanded_channel     = np.zeros((H_e + f, W_e + f));
    for i in range(H_p):
        for j in range(W_p):
            idx = i * stride;
            idy = j * stride;
            expanded_channel[idx : idx + f, idy : idy + f] += np.multiply(paded_channel[i, j], flt)       
    (H_o, W_o) = (H * stride, W * stride)
    output_channel = expanded_channel[int((f + 1)/2) + stride * pad : H_o + int((f + 1)/2) + stride * pad, int((f + 1)/2) + stride * pad : W_o + int((f + 1)/2) + stride * pad]
    
    return output_channel

In [ ]:
def FSNRCNN(mat_contents, img, up_scale):
    weights_conv = mat_contents['weights_conv']
    prelu_conv   = mat_contents['prelu_conv']
    biases_conv  = mat_contents['biases_conv']
    (hei, wid, dpt) = img.shape;
    outhei = hei * up_scale;
    outwid = wid * up_scale;
    layer_num = weights_conv.shape[0];
    in_img = np.true_divide(img, 255)
    im_h = np.zeros((outhei, outwid, dpt))
    for k in range(dpt):
        conv_data = {}
        img_channel = in_img[:, :, k]
        conv_data[-1] = img_channel.reshape((img_channel.shape[0], img_channel.shape[1], 1))

        ## conv
        for idx in range(0, layer_num - 1):
            weight = weights_conv[idx, 0]
            bias   = biases_conv[idx, 0]
            (channels, filtersize, filters) = weight.shape
            patchsize = np.sqrt(filtersize).astype(np.uint8)
            data_tmp  = np.zeros((hei, wid, filters))
            data_pre  = conv_data[idx - 1]
            for i in range(filters): 
                for j in range(channels):
                    subfilter = weight[j, :, i].reshape((patchsize, patchsize))
                    data_tmp[:, :, i] += conv_channel(data_pre[:, :, j], subfilter)
                data_tmp[:, :, i] = np.maximum(data_tmp[:, :, i] + bias[i], 0) + prelu_conv[idx, 0] * np.minimum(data_tmp[:, :, i] + bias[i], 0)
            conv_data[idx] = data_tmp

        ## deconv
        weight = weights_conv[layer_num - 1, 0]
        bias   = biases_conv[layer_num - 1, 0]
        (filters, filtersize, channels) = weight.shape
        temp = np.zeros((channels, filtersize, filters))
        for i in range(channels):
            for j in range(filters):
                temp[i, :, j] = weight[j, :, i]
        weight = temp
        (channels, filtersize, filters) = weight.shape
        patchsize = np.sqrt(filtersize).astype(np.uint8)

        conv3_data = np.zeros((outhei, outwid, 1))
        conv2_data = conv_data[layer_num-2]
        for i in range(filters): 
            for j in range(channels):
                subfilter              = weight[j, :, i].reshape((patchsize, patchsize))
                conv3_data[:, :, i]   += deconv_channel(conv2_data[:, :, j], subfilter, up_scale)
            im_h[:, :, k] = conv3_data[:, :, i] + bias[i]
         
    im_h = np.multiply(im_h, 255)
    for j in range(im_h.shape[0]):
        for i in range(im_h.shape[1]):
            for c in range(im_h.shape[2]):
                if im_h[j, i, c] > 255:
                    im_h[j, i, c] = 255
                if im_h[j, i, c] < 0:
                    im_h[j, i, c] = 0
    im_h = im_h.astype(np.uint8) 
    return im_h

In [ ]:
import time
mat_contents = sio.loadmat('x2.mat')
start_time = time.time()
out_img = FSNRCNN(mat_contents, img, 2) 
stop_time = time.time()
Python_CPU_exec_time = stop_time - start_time
print('FSRCNN Python CPU execution time: ',Python_CPU_exec_time)

In [ ]:
for i in range(out_img.shape[0]):
    for j in range(out_img.shape[1]):
            out_img[i, j, 0], out_img[i, j, 2] =  out_img[i, j, 2], out_img[i, j, 0]
cv2.imwrite('output_image_128.jpeg', out_img)